In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sklearn 
sns.set()

In [ ]:
raw_data = pd.read_excel("CoffeeShop.xlsx",'orders')
def is_valid_item_id(item_id):
    return bool(re.match(r'^It0(0[1-9]|1[0-9]|2[0-4])$', item_id))

# Apply the function to filter valid item ids range(It001 to It024)
data = raw_data[raw_data['item_id'].apply(is_valid_item_id)]
data.info()

In [ ]:
price_data = pd.read_excel("CoffeeShop.xlsx", 'items')
merged_data = pd.merge(price_data, data,on='item_id')
merged_data


In [ ]:
req_data = merged_data[["order_id","item_id"," item_price ","cust_name"]]
req_data[' item_price '] = req_data[' item_price '].replace({'£': ''}, regex=True).astype(float)
req_data

In [ ]:
aggregated_data = req_data.groupby(['order_id', 'cust_name']).agg({' item_price ': 'sum'}).reset_index()
# filtered_data = aggregated_data[aggregated_data['cust_name'] == 'Alex']
# filtered_data
aggregated_data


In [ ]:
customer_data = aggregated_data.groupby('cust_name').agg({
    'order_id': 'count',       # Number of orders
    ' item_price ': 'sum'        # Total spending
}).reset_index()

customer_data['avg']= customer_data[' item_price ']/ customer_data['order_id']
customer_data

In [ ]:
from sklearn.preprocessing import StandardScaler

# Standardize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_data[['order_id', 'avg']])

scaled_features

In [ ]:
from sklearn.cluster import KMeans

wcss=[]

for i in range (1,11):
    kmeans=KMeans(i)
    kmeans.fit(scaled_features)
    wcss.append(kmeans.inertia_)
    
plt.plot(range(1,11), wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Within-cluster Sum of Squares')

In [ ]:
kmeans = KMeans(4)
kmeans.fit(scaled_features)
customer_data['Clusters']= kmeans.fit_predict(scaled_features)
customer_data

In [ ]:
customer_data[['order_id','avg']]= scaler.inverse_transform(scaled_features)


# Plot the clusters
plt.figure(figsize=(12, 8))
sns.scatterplot(data=customer_data, x='order_id', y='avg', hue='Clusters', palette='viridis')
# plt.scatter(customer_data['order_id'], customer_data['avg'], c=customer_data['Clusters'], cmap = 'viridis')

plt.xlabel("Number of Orders")
plt.ylabel("Average Spent")
plt.title("Customer Segments")
plt.show()